In [4]:
import gymnasium as gym
import numpy as np
import loco_mujoco
from loco_mujoco import LocoEnv


env = gym.make("LocoMujoco", env_name="UnitreeA1.simple")
mdp = LocoEnv.make("UnitreeA1.simple.perfect")

# env = gym.make('HalfCheetah-v4')

In [4]:
success = [1.0,1.0,0.0]

print(sum(success)/len(success))

0.6666666666666666


In [6]:
ENV_NAME = 'FetchPush'

ste = f"data/{ENV_NAME}"
ste

'data/FetchPush'

In [5]:
def reformat_dataset(dataset):

    dataset['observations'] = dataset.pop('states')
    dataset['actions'] = dataset.pop('actions')
    dataset['next_observations'] = dataset.pop('next_states')
    dataset['rewards'] = dataset.pop('rewards')
    dataset['terminals'] = dataset.pop('last')

    del dataset['absorbing']

    return dataset

In [6]:
dataset = mdp.create_dataset()
dataset = reformat_dataset(dataset)

In [7]:
dataset['observations'].shape

(25000, 37)

In [8]:
dataset['rewards']

array([0.87187545, 0.84193043, 0.7776974 , ..., 0.88017822, 0.86232611,
       0.82762313])

In [26]:
state

(array([-1.75222744e-01, -7.73589914e-03, -8.11422792e-03,  9.87607437e-04,
         1.83854901e-01,  9.15128876e-01, -2.26819727e+00, -6.37933476e-02,
         9.34214899e-01, -1.91677393e+00, -4.05785975e-02,  9.36312879e-01,
        -1.87045267e+00,  1.60763861e-01,  9.16133264e-01, -2.15280149e+00,
         2.44099928e-01,  3.62274562e-02,  1.94768716e-02,  2.03384910e-01,
         8.99440417e-02, -7.28740115e-03, -8.17263219e-01, -1.93174492e+00,
         2.00346657e+00, -3.39504673e-01,  9.03097584e-01, -2.70741528e-02,
        -3.43999288e-01,  7.30498063e-01,  2.36305918e-01, -5.87035927e-01,
        -2.58242325e+00,  1.63759636e+00,  1.00000000e+00,  0.00000000e+00,
         2.36557250e-01]),
 {})

In [3]:
import numpy as np
from loco_mujoco import LocoEnv
import gymnasium as gym


# create the environment and task together with the reward function
env = gym.make("LocoMujoco", env_name="UnitreeA1.simple.perfect")
# env = gym.wrappers.RecordVideo(env, f"videos/unitreeA1", episode_trigger = lambda x: x % 10 == 0)
action_dim = env.action_space.shape[0]

env.reset()
env.render()
terminated = False
i = 0

while True:
    if i == 1000 or terminated:
        env.reset()
        i = 0
    action = np.random.randn(action_dim)
    nstate, reward, terminated, truncated, info = env.step(action)

    # HERE is your favorite RL algorithm

    env.render()
    i += 1

/home/nikisim/.local/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:335: UserWarning: WARN: No render fps was declared in the environment (env.metadata['render_fps'] is None or not defined), rendering may occur at inconsistent fps.
  logger.warn(


KeyboardInterrupt: 

In [14]:

n_episodes = 5
for _ in range(n_episodes):
    state, done = env.reset(), False
    env.render()
    episode_reward = 0.0
    while not done:
        # action = actor.act(state, device)
        action = env.action_space.sample()
        state, reward, done, _, _ = env.step(action)
        env.render()
        episode_reward += reward
    # episode_rewards.append(episode_reward)
    env.close()
# Each task is associated with a dataset
# dataset contains observations, actions, rewards, terminals, and infos
# dataset = env.get_dataset()
# print(dataset['observations']) # An N x dim_observation Numpy array of observations

# # Alternatively, use d4rl.qlearning_dataset which
# # also adds next_observations.
# dataset = d4rl.qlearning_dataset(env)

/home/nikisim/.local/lib/python3.10/site-packages/gym/envs/mujoco/mujoco_env.py:360: UserWarning: WARN: You are calling render method without specifying any render mode. You can specify the render_mode at initialization, e.g. gym("HalfCheetah-v4", render_mode="rgb_array")
  gym.logger.warn(
/home/nikisim/.local/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


KeyboardInterrupt: 